# VQE of the Z2 lattice gauge theory

In [1]:
# The C2QA package is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import c2qa
import qiskit
import numpy as np
import scipy
import itertools
import c2qa.util as util
import resource
import c2qa.stateReadout as stateReadout
import matplotlib.pyplot as plt
import matplotlib
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ
from collections import Counter

### Prepare initial state and act on it with H1, H2, H3

In [5]:
numberofqubits=2
numberofmodes=3
numberofqubitspermode=2
cutoff=2**numberofqubitspermode

qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)
qbr = qiskit.QuantumRegister(size=numberofqubits)
cbr = qiskit.ClassicalRegister(size=1) # To read out the results of the Hadamard test
circuit = c2qa.CVCircuit(qmr, qbr, cbr)

diffstallmodes=[2,2,2]
for i in range(qmr.num_qumodes):
    circuit.cv_initialize(diffstallmodes[i], qmr[i])

for i in range(numberofqubits):
    circuit.h(qbr[i])

stateop, _ = c2qa.util.simulate(circuit)
occ=util.stateread(stateop, qbr.size, numberofmodes, cutoff)

theta_1=np.pi/2
theta_2=np.pi/2
theta_3=np.pi/2

# resource.vary_Z2LGT(circuit, numberofmodes, qmr, qbr, theta_1, theta_2, theta_3)
# circuit.barrier()
# for i in range(numberofqubits):
#     resource.measureE_fieldterm(circuit, qbr, i)
# for i in range(numberofmodes-1):
resource.vary_Z2LGT(circuit, numberofmodes, qmr, qbr, theta_1, theta_2, theta_3)
circuit.barrier()
resource.measureE_hoppingterm(circuit, numberofmodes, numberofqubits, qmr, qbr, 0)
circuit.barrier()
for i in range(numberofqubits):
    resource.measureE_fieldterm(circuit, qbr, i)

circuit.draw(output='mpl', filename='VQE_resourceH_measureE.png')

stateop, result = c2qa.util.simulate(circuit)
# util.stateread(stateop, qbr.size, numberofmodes, 4)
print('Counts(ideal):', result.get_counts())

qumodes:  222  qubits:  00     with amplitude:  0.5000000000000001
qumodes:  222  qubits:  01     with amplitude:  0.5
qumodes:  222  qubits:  10     with amplitude:  0.5
qumodes:  222  qubits:  11     with amplitude:  0.4999999999999999
occupation modes  [2.0, 2.0, 2.0]
occupation qubits  [0.4999999999999999, 0.4999999999999999]
Counts(ideal): {'0': 1024}


In [6]:
circuit.draw(output='mpl', filename='VQE_resourceH_measureE.png')

<Figure size 3323.87x1571.89 with 1 Axes>